In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
import random
from mlxtend.frequent_patterns import apriori
import pyfpgrowth
from format_rules import format_rules
import boto3
from io import BytesIO

In [13]:
df2=pd.read_csv('df_test.csv')

In [ ]:
client = boto3.client('s3')
obj = client.get_object(Bucket='manifolddata', Key='week1.csv')
df = pd.read_csv(BytesIO(obj['Body'].read()), low_memory=False)

In [11]:
df=df.iloc[:,[0,1,3,4,5,6,7,8]]
df.columns=['Date', 'Duration', 'Src_IP', 'Src_pt', 'Dst_IP', 'Dst_pt','Packets', 'Bytes']

df['Date']=pd.to_datetime(df['Date'], format="%Y-%m-%d %H:%M:%S.%f", errors = 'coerce')
df['date_hr']=pd.Series(df['Date']).dt.round("H")
#create a pair column, which is a touple of the src and dst IP, sorted. 
#It does not matter which call came first, we simply want to know which pair occurs most frequently.

df['pairs']=list(zip(df.Src_IP, df.Dst_IP))
df['pairs']=df['pairs'].apply(sorted)
df['pairs2']=tuple(df['pairs'])

def convert_si_to_number(x):
    total_stars = 0
    if 'k' in x:
        if len(x) > 1:
            total_stars = float(x.replace('k', '')) * 1000 # convert k to a thousand
    elif 'M' in x:
        if len(x) > 1:
            total_stars = float(x.replace('M', '')) * 1000000 # convert M to a million
    elif 'B' in x:
        total_stars = float(x.replace('B', '')) * 1000000000 # convert B to a Billion
    else:
        total_stars = int(x) # Less than 1000
    return int(total_stars)


df.Bytes=df.Bytes.astype('str')
test_list=df.Bytes

[i for i, s in enumerate(test_list) if 'M' in s]#show where the M errors are happening

test_list= [convert_si_to_number(x) for x in test_list]
df.Bytes=test_list #bring it back into the dataframe

#create a normalized latency column = duration/packets
df['norm_latency']=df['Duration']/df['Packets']

#group the dataset by unique pairs, then count how frequently each pair occurs, and also get the average normalized latency time for that pair. The latency times vary because the servers they are assigned are random.
pairs_count=(df.groupby('pairs2').agg({'Date':'count', 'norm_latency': 'mean', 'Duration': 'sum', 'Packets':'sum'}).reset_index())

pairs_count.columns=['pairs','frequency', 'avg_norm_latency', 'total_duration', 'total_packets']
pairs_count['norm_latency']=(pairs_count['total_duration']/pairs_count['total_packets'].sum())*100 #sum of all duration time divided by sum of all packets transfered for that pair

#we only want a list of all the individual pairs at each timestamp. Think of this where each timestamp is a 'transaction' and we chose to buy which 2 items (IP addresses)
data_l=list(df['pairs'])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stabl

In [14]:
df2.head()

,Unnamed: 0,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2,norm_latency
0,0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02 00:00:00.000,"['192.168.100.11', '192.168.210.55']","('192.168.100.11', '192.168.210.55')",0.0015
1,1,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02 00:00:00.000,"['192.168.100.11', '192.168.210.55']","('192.168.100.11', '192.168.210.55')",0.0000
2,2,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02 00:00:00.000,"['192.168.100.11', '192.168.220.47']","('192.168.100.11', '192.168.220.47')",0.0020
3,3,2017-08-02 00:00:02.859,0.000,10000_34,443,192.168.210.54,59628.0,1,100,2017-08-02 00:00:00.000,"['10000_34', '192.168.210.54']","('10000_34', '192.168.210.54')",0.0000
4,4,2017-08-02 00:00:02.594,0.000,192.168.100.11,445,192.168.220.47,55101.0,1,108,2017-08-02 00:00:00.000,"['192.168.100.11', '192.168.220.47']","('192.168.100.11', '192.168.220.47')",0.0000


In [24]:
pairs_count2=(df2.groupby('pairs2').agg({'Date':'count', 'norm_latency': 'mean', 'Duration': 'sum', 'Packets':'sum'}).reset_index())
pairs_count2.columns=['pairs','frequency', 'avg_norm_latency', 'total_duration', 'total_packets']
pairs_count2['norm_latency']=(pairs_count2['total_duration']/pairs_count2['total_packets'].sum())*100 #su
pairs_count2.head()

,pairs,frequency,avg_norm_latency,total_duration,total_packets,norm_latency
0,"('10000_34', '192.168.210.54')",6,0.062833,1.082,14,0.001058
1,"('10001_213', '192.168.210.54')",109,0.003924,2.344,291,0.002293
2,"('10002_162', '192.168.210.57')",10,0.034261,0.805,38,0.000787
3,"('10002_174', '192.168.210.54')",50,0.042842,6.245,190,0.006109
4,"('10002_174', '192.168.210.57')",70,0.043185,8.979,241,0.008783


In [15]:
df.head()

,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2,norm_latency
0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",0.0015
1,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",0.0000
2,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",0.0020
3,2017-08-02 00:00:02.859,0.000,10000_34,443,192.168.210.54,59628.0,1,100,2017-08-02,"[10000_34, 192.168.210.54]","(10000_34, 192.168.210.54)",0.0000
4,2017-08-02 00:00:02.594,0.000,192.168.100.11,445,192.168.220.47,55101.0,1,108,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",0.0000


In [22]:
pairs_count=(df.groupby('pairs2').agg({'Date':'count', 'norm_latency': 'mean', 'Duration': 'sum', 'Packets':'sum'}).reset_index())
pairs_count.columns=['pairs','frequency', 'avg_norm_latency', 'total_duration', 'total_packets']
pairs_count['norm_latency']=(pairs_count['total_duration']/pairs_count['total_packets'].sum())*100 #su
pairs_count.head()

,pairs2,Date,norm_latency,Duration,Packets
0,"(0.0.0.0, 255.255.255.255)",123,0.018820,6.801,223
1,"(10000_163, 192.168.220.47)",16,0.026500,6.378,204
2,"(10000_34, 192.168.210.45)",96,0.022706,23.679,723
3,"(10000_34, 192.168.210.46)",165,0.014315,28.699,971
4,"(10000_34, 192.168.210.47)",433,0.020122,77.542,1739


,pairs,frequency,avg_norm_latency,total_duration,total_packets,norm_latency
0,"(0.0.0.0, 255.255.255.255)",123,0.018820,6.801,223,0.000004
1,"(10000_163, 192.168.220.47)",16,0.026500,6.378,204,0.000003
2,"(10000_34, 192.168.210.45)",96,0.022706,23.679,723,0.000013
3,"(10000_34, 192.168.210.46)",165,0.014315,28.699,971,0.000016
4,"(10000_34, 192.168.210.47)",433,0.020122,77.542,1739,0.000042


In [25]:
per_40=np.percentile(pairs_count['frequency'], [40])[0]
per_80=np.percentile(pairs_count['frequency'], [80])[0]
#patterns40 = pyfpgrowth.find_frequent_patterns(data_l, per_40) 
patterns80 = pyfpgrowth.find_frequent_patterns(data_l, per_80) #generates less patterns b/c the min threshold is set higher

confidence=0.7 #this means the rule is likely to be true 70% of the time, it is a high threshold, used for testing
#rules40 = pyfpgrowth.generate_association_rules(patterns40, confidence)
rules80 = pyfpgrowth.generate_association_rules(patterns80, confidence)
#the input 'rules' is the result of the pyfgrowth function run above. You must have specified the thresholds you want to use
#the orig_df is the original data, with 

In [26]:
rules_df_80per_70con=format_rules(rules80, df, 20)

In [27]:
rules_df_80per_70con.head()

,IP_A,IP_B,confidence,pairs,pairs2,frequency,avg_norm_latency,total_duration,total_packets,norm_latency,latency_rank,pair_ID,server_A,server_B,server
0,13053_118,192.168.210.56,1.0,"[13053_118, 192.168.210.56]","(13053_118, 192.168.210.56)",12439,0.255143,15736.067,44427,0.008610,107.096797,0,0,0,0
1,13049_193,192.168.210.56,1.0,"[13049_193, 192.168.210.56]","(13049_193, 192.168.210.56)",1775,0.372031,79345.558,245786,0.043413,77.057656,1,None,None,0
2,13053_106,192.168.210.56,1.0,"[13053_106, 192.168.210.56]","(13053_106, 192.168.210.56)",7180,0.343261,14717.209,31305,0.008052,57.815558,2,None,None,0
3,10272_44,192.168.220.51,1.0,"[10272_44, 192.168.220.51]","(10272_44, 192.168.220.51)",10094,0.091117,3665.353,26275,0.002005,20.242956,3,None,None,0
4,14170_51,192.168.220.51,1.0,"[14170_51, 192.168.220.51]","(14170_51, 192.168.220.51)",8094,0.022052,1881.134,26460,0.001029,8.330628,4,None,None,0


EVERYTHING ABOVE WORKS, TRY EXACT SAME BUT WITH DIFFERENT INPUT DATA

In [28]:
len(rules_df_80per_70con)

1671